# COVID-Net USPro: Prototypical Network on Ultrasound Images for COVID-19 Detection

This notebook shows the basic usage of COVID-Net USPro. A subset of testing data (unexposed to trained network) and trained networks for 5 and 100 shots conditions are used for the demonstration. 

- Testing data: /data_sample_test 
- Trained networks: /protonet/save_models
- Outputs for examples stored in: /protonet/protonet_example_results 

In [1]:
import numpy as np

In [3]:
## Settings
test_n_way = 2 ## change to 3 or 4 if performing 3-way or 4-way classification
test_n_support = 5
test_n_query = 5
test_episode = 200

In [4]:
from loader.load_data import read_images

## Load dataset
test_directory = '../data_sample_test/'
test_x, test_y = read_images(test_directory)  

In [5]:
from models.ProtoNet import load_protonet_conv

encoder = 2
prob_type = 2

model = load_protonet_conv(encoder, prob_type)

In [7]:
import torch

model_path = './save_models/model1_shot5_ways2.pth'

if torch.cuda.is_available():
    model.load_state_dict(torch.load(model_path)) 
else: 
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))

In [8]:
model

ProtoNet(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [9]:
from models.test_model import test

batch_size = 60
test(model, prob_type, batch_size, test_x, test_y, test_n_way, test_n_support, test_n_query, test_episode)

  0%|          | 0/200 [00:00<?, ?it/s]

-- Loss: 0.6810 Acc: 0.8885
class 0 -- Precision: 0.8976 Recall: 0.9020
class 1 -- Precision: 0.9180 Recall: 0.8750
[[902.  98.]
 [125. 875.]]


## GradCAM examples
- Produce GradCAM analysis for classified results such as false positives and true positives

In [10]:
from torchvision.models import resnet18
from torch import nn
model_1 = resnet18()
num_ftrs = model_1.fc.in_features
model_1.fc = nn.Linear(num_ftrs, prob_type) # reinitialize last layer to have out_feature, corresponding to num_class

model_en_path = './save_models/model1_shot5_ways2_encoder.pth'

## load trained encoder parameters 
if torch.cuda.is_available():
    model_1.load_state_dict(torch.load(model_en_path)) 
else: 
    model_1.load_state_dict(torch.load(model_en_path, map_location=torch.device("cpu")))

In [11]:
import warnings; warnings.simplefilter('ignore')
from models.test_model_gradcam import test_gradcam

n_support = 5
n_query = 1
n_way = 2 
episode = 100

true_pos, true_pos_possibility, true_neg, true_neg_possibility, false_pos, false_neg = test_gradcam(model, test_x, test_y, n_way, n_support, n_query, episode)

  0%|          | 0/100 [00:00<?, ?it/s]

true_pos.shape torch.Size([86, 3, 224, 224])
true_neg.shape torch.Size([91, 3, 224, 224])
false_pos.shape torch.Size([9, 3, 224, 224])
false_neg.shape torch.Size([12, 3, 224, 224])
-- Loss: 0.7707 Acc: 0.8850
class 0 -- Precision: 0.8500 Recall: 0.9100
class 1 -- Precision: 0.8250 Recall: 0.8600


In [12]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

def get_visualization_gradcam(model, input, index):
    target_layers = [model.layer4[-1]]
    input_tensor = input / 255
    input_tensor = input_tensor.float()
    rgb_img = input_tensor[index].permute(1,2,0).cpu().detach().numpy()

    # Construct the CAM object once, and then re-use it on many images:
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
    grayscale_cam = cam(input_tensor=input_tensor, targets=None)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[index, :]
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    return visualization

In [13]:
import cv2
def get_images(encoder, image_array, array_type):
    for i in range(0, image_array.shape[0]): 
        visualization = get_visualization_gradcam(encoder, image_array, i)
        cam_image = cv2.cvtColor(visualization, cv2.COLOR_RGB2BGR)
        cam_path = f'./{array_type}_{i}.jpg'
        cv2.imwrite(cam_path, cam_image)

        orig = image_array[i].permute(1,2,0).cpu().detach().numpy()
        orig_image = cv2.cvtColor(orig, cv2.COLOR_RGB2BGR)
        orig_path = f'./{array_type}_original_{i}.jpg'
        cv2.imwrite(orig_path, orig_image)
    print(image_array.shape[0])

In [14]:
get_images(model_1, false_neg[:3,:,:,:], "./protonet_example_results/false_negative")

3


In [15]:
get_images(model_1, false_pos[:3,:,:,:], "./protonet_example_results/false_positive")

3
